Udacity deep learning [assignment 2](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb)

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


### Logistic Regression using tf

In [4]:
batch_size = 128

In [5]:
graph_LR = tf.Graph()
with graph_LR.as_default():
#     tf_train_dataset = tf.constant(train_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    x = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.variable_scope("LogisticRegression"):
        W = tf.get_variable("weights", (image_size*image_size,num_labels),initializer=tf.random_uniform_initializer())
        b = tf.get_variable("bias",(num_labels,), initializer=tf.constant_initializer(0.0))
        biases = tf.Variable(tf.zeros([num_labels]))
        print(x.get_shape(),W.get_shape(),b.get_shape(), biases.get_shape())
        logits = tf.matmul(x, W) + b
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
#         train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, W) + b)
        valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, W) + b)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, W) + b)

(128, 784) (784, 10) (10,) (10,)


In [6]:
def accuracy(pred, labels):
    return (100.0 * np.sum(np.argmax(pred,axis=1) == np.argmax(labels,axis=1)) / pred.shape[0])

In [7]:
num_steps = 801

In [8]:
with tf.Session(graph=graph_LR) as sess:
    sess.run(tf.initialize_all_variables())
    print("Initialized .")
    for i in xrange(num_steps):
        indices = np.random.choice(train_dataset.shape[0], batch_size)
        x_batch = train_dataset[indices]
        y_batch = train_labels[indices]
        _, l = sess.run([optimizer, loss], feed_dict={x:x_batch, y:y_batch})
        if i % 100 == 0:
            print("Loss at step %i = %.5f" %(i, l))
            #t_pred = train_prediction.eval()
            v_pred = valid_prediction.eval()
#             print("Training accuracy = %.2f, Validation accuracy = %.2f" \
#                   %(accuracy(t_pred,train_labels), accuracy(v_pred, valid_labels)))
            print("Validation accuracy = %.2f" %(accuracy(v_pred, valid_labels)))
    print("Completed training .")
    test_pred = test_prediction.eval()
    print("Test accuracy = %.2f" % accuracy(test_pred, test_labels))
        

Initialized .
Loss at step 0 = 6.99332
Validation accuracy = 15.84
Loss at step 100 = 1.17128
Validation accuracy = 76.51
Loss at step 200 = 0.72741
Validation accuracy = 78.55
Loss at step 300 = 0.88049
Validation accuracy = 78.46
Loss at step 400 = 0.91479
Validation accuracy = 79.77
Loss at step 500 = 0.93101
Validation accuracy = 80.07
Loss at step 600 = 0.74982
Validation accuracy = 80.63
Loss at step 700 = 0.73281
Validation accuracy = 80.68
Loss at step 800 = 0.76789
Validation accuracy = 81.32
Completed training .
Test accuracy = 87.59
